In [22]:
import numpy as np
import matplotlib.pyplot as plt 

%matplotlib inline

In [23]:
from random import randint
from numpy import array
from numpy import argmax
from numpy import array_equal
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense

Using TensorFlow backend.


In [24]:
# generate a sequence of random integers
def generate_sequence(length, n_unique):
	return [randint(1, n_unique-1) for _ in range(length)]

In [33]:
def get_dataset(n_in, n_out, cardinality, n_samples):
        X1, X2, y = list(), list(), list()
        for _ in range(n_samples):
                # generate source sequence
                source = generate_sequence(n_in, cardinality)
                # define padded target sequence
                target = source[:n_out]
                target.reverse()
                # create padded input target sequence
                target_in = [0] + target[:-1]
                # encode
                src_encoded = to_categorical([source], num_classes=cardinality)
                tar_encoded = to_categorical([target], num_classes=cardinality)
                tar2_encoded = to_categorical([target_in], num_classes=cardinality)
                # store
                X1.append(src_encoded)
                X2.append(tar2_encoded)
                y.append(tar_encoded)
        X1 = np.squeeze(array(X1), axis=1) 
        X2 = np.squeeze(array(X2), axis=1) 
        y = np.squeeze(array(y), axis=1) 
        return X1, X2, y

In [26]:
# returns train, inference_encoder and inference_decoder models
def define_models(n_input, n_output, n_units):
	# define training encoder
	encoder_inputs = Input(shape=(None, n_input))
	encoder = LSTM(n_units, return_state=True)
	encoder_outputs, state_h, state_c = encoder(encoder_inputs)
	encoder_states = [state_h, state_c]
	# define training decoder
	decoder_inputs = Input(shape=(None, n_output))
	decoder_lstm = LSTM(n_units, return_sequences=True, return_state=True)
	decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
	decoder_dense = Dense(n_output, activation='softmax')
	decoder_outputs = decoder_dense(decoder_outputs)
	model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
	# define inference encoder
	encoder_model = Model(encoder_inputs, encoder_states)
	# define inference decoder
	decoder_state_input_h = Input(shape=(n_units,))
	decoder_state_input_c = Input(shape=(n_units,))
	decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
	decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
	decoder_states = [state_h, state_c]
	decoder_outputs = decoder_dense(decoder_outputs)
	decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
	# return all models
	return model, encoder_model, decoder_model

In [27]:
# generate target given source sequence
def predict_sequence(infenc, infdec, source, n_steps, cardinality):
	# encode
	state = infenc.predict(source)
	# start of sequence input
	target_seq = array([0.0 for _ in range(cardinality)]).reshape(1, 1, cardinality)
	# collect predictions
	output = list()
	for t in range(n_steps):
		# predict next char
		yhat, h, c = infdec.predict([target_seq] + state)
		# store prediction
		output.append(yhat[0,0,:])
		# update state
		state = [h, c]
		# update target sequence
		target_seq = yhat
	return array(output)

In [28]:
# decode a one hot encoded string
def one_hot_decode(encoded_seq):
	return [argmax(vector) for vector in encoded_seq]

In [40]:
# configure problem
n_features = 50 + 1
n_steps_in = 6
n_steps_out = 3
# define model
train, infenc, infdec = define_models(n_features, n_features, 128)
train.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])



# generate training dataset
X1, X2, y = get_dataset(n_steps_in, n_steps_out, n_features, 10000)
print(X1.shape,X2.shape,y.shape)
# train model

(10000, 6, 51) (10000, 3, 51) (10000, 3, 51)


In [41]:
train.fit([X1, X2], y, epochs=10)
# evaluate LSTM
total, correct = 100, 0

Epoch 1/10
10000/10000 [==============================] - 5s 476us/step - loss: 2.6223 - acc: 0.2585
Epoch 2/10
10000/10000 [==============================] - 3s 321us/step - loss: 1.5882 - acc: 0.4207
Epoch 3/10
10000/10000 [==============================] - 3s 324us/step - loss: 1.2136 - acc: 0.5366
Epoch 4/10
10000/10000 [==============================] - 3s 338us/step - loss: 0.8626 - acc: 0.6865
Epoch 5/10
10000/10000 [==============================] - 3s 320us/step - loss: 0.5071 - acc: 0.8381
Epoch 6/10
10000/10000 [==============================] - 3s 319us/step - loss: 0.2452 - acc: 0.9385
Epoch 7/10
10000/10000 [==============================] - 3s 324us/step - loss: 0.1130 - acc: 0.9795
Epoch 8/10
10000/10000 [==============================] - 3s 320us/step - loss: 0.0528 - acc: 0.9927
Epoch 9/10
10000/10000 [==============================] - 3s 321us/step - loss: 0.0248 - acc: 0.9974
Epoch 10/10
10000/10000 [==============================] - 3s 324us/step - loss: 0.0127 - a

In [42]:
for _ in range(total):
	X1, X2, y = get_dataset(n_steps_in, n_steps_out, n_features, 1)
	target = predict_sequence(infenc, infdec, X1, n_steps_out, n_features)
	if array_equal(one_hot_decode(y[0]), one_hot_decode(target)):
		correct += 1
print('Accuracy: %.2f%%' % (float(correct)/float(total)*100.0))
# spot check some examples
for _ in range(10):
	X1, X2, y = get_dataset(n_steps_in, n_steps_out, n_features, 1)
	target = predict_sequence(infenc, infdec, X1, n_steps_out, n_features)
	print('X=%s y=%s, yhat=%s' % (one_hot_decode(X1[0]), one_hot_decode(y[0]), one_hot_decode(target)))

Accuracy: 99.00%
X=[3, 5, 37, 10, 19, 20] y=[37, 5, 3], yhat=[37, 5, 3]
X=[31, 6, 49, 8, 8, 29] y=[49, 6, 31], yhat=[49, 6, 31]
X=[36, 45, 5, 46, 37, 31] y=[5, 45, 36], yhat=[5, 45, 36]
X=[21, 10, 21, 50, 44, 31] y=[21, 10, 21], yhat=[21, 10, 21]
X=[9, 17, 25, 38, 27, 1] y=[25, 17, 9], yhat=[25, 17, 9]
X=[4, 46, 32, 11, 1, 23] y=[32, 46, 4], yhat=[32, 46, 4]
X=[46, 49, 8, 4, 16, 37] y=[8, 49, 46], yhat=[8, 49, 46]
X=[1, 32, 49, 50, 36, 16] y=[49, 32, 1], yhat=[49, 32, 1]
X=[46, 18, 5, 27, 41, 19] y=[5, 18, 46], yhat=[5, 18, 46]
X=[13, 10, 39, 43, 10, 41] y=[39, 10, 13], yhat=[39, 10, 13]
